# **Librerías**

In [4]:
# Cargar librerías
import os, mlflow
from dotenv import load_dotenv
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature
from mlflow import MlflowClient
import mlflow

# **Establecer el experimento**


In [10]:
load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/danydelatorregallo@gmail.com/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

# **Preparar los datos**


In [14]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

df_train = read_dataframe('C:\\Users\\Daniela de la Torre\\OneDrive\\Escritorio\\Universidad\\5to_Semestre\\ProyectoCD\\nyc-taxi-predictions-2025-2\\data\\green_tripdata_2025-01.parquet')
df_val = read_dataframe('C:\\Users\\Daniela de la Torre\\OneDrive\\Escritorio\\Universidad\\5to_Semestre\\ProyectoCD\\nyc-taxi-predictions-2025-2\\data\\green_tripdata_2025-02.parquet')

In [15]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

# **Gradient Boost**
Usando el código de clase hacemos los trials de ambos modelos

In [17]:
from sklearn.ensemble import GradientBoostingRegressor

def objective_gb(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 100),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "random_state": 42
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "gradient_boosting")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento 
        model = GradientBoostingRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val[:5].toarray(), y_pred[:5])

         # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out()),
            signature=signature
        )      

    # Optuna minimiza el valor retornado
    return rmse

In [18]:
mlflow.sklearn.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_gb = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------

with mlflow.start_run(run_name="Gradient Boosting Hyperparameter Optimization (Optuna)", nested=True):
    study_gb.optimize(objective_gb, n_trials=3)

        # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study_gb.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "gradient_boosting",
        "feature_set_version": 1,
    })

     # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    
    model = GradientBoostingRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out())
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.sklearn.log_model(model, name="model", input_example=input_example, signature=signature)

[I 2025-10-28 18:50:45,311] A new study created in memory with name: no-name-ce310dde-0889-42d8-ae0a-90dfdf69f618
2025/10/28 18:51:46 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 18:51:50 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 18:51:58,555] Trial 0 finished with value: 5.423033869318215 and parameters: {'n_estimators': 69, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_samples_split': 7, 'min_samples_leaf': 2}. Best is trial 0 with value: 5.423033869318215.


🏃 View run luxuriant-snipe-882 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/9b38a642bc5640199b6cffc116cecb04
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


2025/10/28 18:52:27 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 18:52:29 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 18:52:40,157] Trial 1 finished with value: 6.667930901959873 and parameters: {'n_estimators': 57, 'learning_rate': 0.012184186502221764, 'max_depth': 9, 'min_samples_split': 7, 'min_samples_leaf': 8}. Best is trial 0 with value: 5.423033869318215.


🏃 View run bemused-smelt-654 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/9758ff91ce924b9a8162f26ffa33e94a
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


2025/10/28 18:53:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 18:53:20 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 18:53:26,939] Trial 2 finished with value: 5.418235873807679 and parameters: {'n_estimators': 51, 'learning_rate': 0.2708160864249968, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 2 with value: 5.418235873807679.


🏃 View run handsome-mole-612 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/03bb2162ef4a4e7081401d6d6cbcffb8
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


2025/10/28 18:54:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Daniela de la Torre\OneDrive\Escritorio\Universidad\5to_Semestre\ProyectoCD\nyc-taxi-predictions-2025-2\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
2025/10/28 18:54:30 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Gradient Boosting Hyperparameter Optimization (Optuna) at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/80349b633c454e878a16938233456065
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


# **Random Forest**

In [19]:
from sklearn.ensemble import RandomForestRegressor

mlflow.sklearn.autolog(log_models=False)

def objective_rf(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 75),
        "max_depth": trial.suggest_int("max_depth", 5, 15),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 8),
        "random_state": 42,
        "n_jobs": -1
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "random_forest")
        mlflow.log_params(params)

        # Entrenamiento 
        model = RandomForestRegressor(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val[:5].toarray(), y_pred[:5])
        
        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.sklearn.log_model(
            model,
            name="model",
            input_example=pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out()),
            signature=signature
        )

    return rmse

In [20]:
# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study_rf = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------

with mlflow.start_run(run_name="Random Forest Hyperparameter Optimization (Optuna)", nested=True):
    study_rf.optimize(objective_rf, n_trials=4)

    # Recuperar y registrar los mejores hiperparámetros
    best_params = study_rf.best_params
    best_params["random_state"] = 42

    mlflow.log_params(best_params)
    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "random_forest",
        "feature_set_version": 1,
    })

     # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    
    model = RandomForestRegressor(**best_params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

     # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=dv.get_feature_names_out())
    signature = infer_signature(input_example, y_val[:5])

    mlflow.sklearn.log_model(model, name="model", input_example=input_example, signature=signature)

[I 2025-10-28 18:55:56,148] A new study created in memory with name: no-name-a22fb9cb-8494-4124-bb0a-6a89a5b6ef6a
2025/10/28 18:57:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 18:57:39 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run stylish-swan-693 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/bdcd2dd758d746a08019691beac37ea2
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


[I 2025-10-28 19:00:04,876] Trial 0 finished with value: 5.51368908987859 and parameters: {'n_estimators': 59, 'max_depth': 15, 'min_samples_split': 8, 'min_samples_leaf': 5}. Best is trial 0 with value: 5.51368908987859.
2025/10/28 19:00:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 19:00:43 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
[I 2025-10-28 19:00:52,731] Trial 1 finished with value: 5.619616128247637 and parameters: {'n_estimators': 54, 'max_depth': 6, 'min_samples_split': 2, 'min_samples_leaf': 7}. Best is trial 0 with value: 5.51368908987859.


🏃 View run polite-colt-583 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/17b11296fd8848f48e433b2a7a63526d
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


2025/10/28 19:01:24 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 19:01:27 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run bright-trout-858 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/85034b01f30f41138d88626280dd68ff
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


[I 2025-10-28 19:01:42,673] Trial 2 finished with value: 5.568737521190013 and parameters: {'n_estimators': 65, 'max_depth': 12, 'min_samples_split': 2, 'min_samples_leaf': 8}. Best is trial 0 with value: 5.51368908987859.
2025/10/28 19:02:04 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/28 19:02:06 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run big-snail-541 at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/c47e4bcebfbe4ea796e050cc0c0e7b4f
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


[I 2025-10-28 19:02:14,829] Trial 3 finished with value: 5.60256347819403 and parameters: {'n_estimators': 71, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 0 with value: 5.51368908987859.
2025/10/28 19:02:52 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
c:\Users\Daniela de la Torre\OneDrive\Escritorio\Universidad\5to_Semestre\ProyectoCD\nyc-taxi-predictions-2025-2\.venv\Lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
2025/10/28 19:02:58 INFO mlflow.models.model: Found the following environment variables used during model inference: [DATABRICKS_HOST, DATABRICKS_TOKEN]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.


🏃 View run Random Forest Hyperparameter Optimization (Optuna) at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437/runs/944ee784c38a41fbb361e21a2391f2a7
🧪 View experiment at: https://dbc-4bc8764d-9eba.cloud.databricks.com/ml/experiments/1027002703688437


# **Guardar el modelo en el Model Registry**
El mejor modelo fue Grading con un RMSE de 5.1418 así que ese erá el que usemos de challenger:

In [21]:
client = MlflowClient()
model_name = "workspace.default.nyc-taxi-model"
run_id = '80349b633c454e878a16938233456065'

Registramos el modelo

In [22]:
# Registrar el mejor modelo
result = mlflow.register_model(
    model_uri=f"runs:/{run_id}/model",
    name=model_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/28 19:06:53 WARNING mlflow.tracking._model_registry.fluent: Run with id 80349b633c454e878a16938233456065 has no artifacts at artifact path 'model', registering model based on models:/m-40a2586a59dc46f783b7d217dddb5dc3 instead
Uploading artifacts: 100%|██████████| 9/9 [00:05<00:00,  1.58it/s]
Created version '3' of model 'workspace.default.nyc-taxi-model'.


In [23]:
model_version = result.version
new_alias = "Challenger"

client.set_registered_model_alias(
    name=model_name,
    alias=new_alias,
    version=result.version
)

# **Champion vs Challenger**
Hay que cargar los modelos para ver cuál es mejor:

In [24]:
import mlflow.pyfunc
champ_uri = f"models:/{model_name}@Champion"
chall_uri = f"models:/{model_name}@Challenger"

champ_version = mlflow.pyfunc.load_model(champ_uri)
chall_version = mlflow.pyfunc.load_model(chall_uri)

2025/10/28 19:07:28 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - psutil (current: 7.1.2, required: psutil==7.1.1)
 - scipy (current: 1.16.3, required: scipy==1.16.2)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.
c:\Users\Daniela de la Torre\OneDrive\Escritorio\Universidad\5to_Semestre\ProyectoCD\nyc-taxi-predictions-2025-2\.venv\Lib\site-packages\mlflow\xgboost\__init__.py:321: UserWarning: [19:07:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


Preparamos la data:

In [28]:
df_val = read_dataframe(r'C:\Users\Daniela de la Torre\OneDrive\Escritorio\Universidad\5to_Semestre\ProyectoCD\nyc-taxi-predictions-2025-2\data\green_tripdata_2025-03.parquet')
X_val = preprocess(df_val, dv)
target = 'duration'
y_val = df_val[target].values

In [31]:
chall_version.predict(X_val)

MlflowException: Failed to enforce schema of data '<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 93697 stored elements and shape (48336, 4159)>
  Coords	Values
  (0, 3391)	1.0
  (0, 4158)	2.2
  (1, 2497)	1.0
  (1, 4158)	1.06
  (2, 4158)	18.91
  (3, 3717)	1.0
  (3, 4158)	8.36
  (4, 3072)	1.0
  (4, 4158)	0.82
  (5, 4158)	4.94
  (6, 2150)	1.0
  (6, 4158)	0.81
  (7, 1798)	1.0
  (7, 4158)	1.56
  (8, 3412)	1.0
  (8, 4158)	1.53
  (9, 3996)	1.0
  (9, 4158)	4.56
  (10, 3582)	1.0
  (10, 4158)	0.89
  (11, 4041)	1.0
  (11, 4158)	1.17
  (12, 3953)	1.0
  (12, 4158)	2.48
  (13, 4158)	2.65
  :	:
  (48323, 1762)	1.0
  (48323, 4158)	9.1
  (48324, 2497)	1.0
  (48324, 4158)	1.54
  (48325, 642)	1.0
  (48325, 4158)	1.51
  (48326, 4158)	8.28
  (48327, 758)	1.0
  (48327, 4158)	3.3
  (48328, 3097)	1.0
  (48328, 4158)	2.25
  (48329, 799)	1.0
  (48329, 4158)	1.09
  (48330, 798)	1.0
  (48330, 4158)	1.92
  (48331, 2038)	1.0
  (48331, 4158)	1.57
  (48332, 3275)	1.0
  (48332, 4158)	2.31
  (48333, 300